In [1]:
from glob import glob
import pandas as pd
import numpy as np
from collections import Counter
from IPython.display import display, HTML
import re

In [2]:
# Base directories
BASE_LICITACOES = "../data/dados_abertos/"

In [29]:
# Lists all *.csv files
csv_files = glob(BASE_LICITACOES + "/*/*/*.csv", recursive=True)

# Select document with at least one data, ie, more than one line
csv_file_with_content = list()
for csv_file in csv_files:
    # if "itemLicitacao" not in csv_file:
    #    continue
        
    with open(csv_file) as f:
        lines = f.readlines()
        if len(lines) > 1 and ("seq_item_licitacao" in lines[0] or "seq_licitacao" in lines[0] or "seq_orgao" in lines[0] or "cod_item" in lines[0]):
            csv_file_with_content.append(csv_file)

n = len(csv_file_with_content)
total = len(csv_files)
print(f"{n} with any content that correspond to {100*n/total}%")
        

100684 with any content that correspond to 76.15172257308173%


In [4]:
csv_file_with_content[:10]

['../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.cotacaoLicitacao.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.fornDispensa.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.itemLicitacao.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.dispensa.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.recLicitacao.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.regadesao.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.credDispensa.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.homologLicitacao.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.julgLicitacao.csv',
 '../data/dados_abertos/licitacao_2020/3101409/2020.3101409.licitacao.respLicitacao.csv']

In [ ]:
# Count diferent headers and fill a single data frame
counter = dict()
content = []
for csv_file in csv_file_with_content:
    filename = csv_file.split("/")[-1]
    with open(csv_file) as f:
        lines = f.readlines()
        header = lines[0].split(";")        
        
        for h in header:
            if h not in counter:
                counter[h] = {"header": h, "counts": 0, "files": filename}
                
            counter[h]["counts"] = counter[h]["counts"] + 1
            
            if counter[h]["files"] != filename:
                counter[h]["files"] += ", " + filename
            
            for line in lines[1:]:
                dd = line.split(";")
                record = dict(zip(header, dd))
                content.append(record)
        

_header = pd.DataFrame.from_records(list(counter.values()))
_df = pd.DataFrame.from_records(content)

In [25]:
_header.to_csv("output_dados_abertos/headers.csv")

In [28]:
_df.to_csv("output_dados_abertos/results.csv", index=False)

In [22]:
_df.loc[_df.num_item.notna(), ["num_item", "dsc_item"]]

,num_item,dsc_item
680,1,"""KIT ALIMENTACAO PARA ALUNOS CONTENDO:"""
681,7,"""SOLUCAO TAMPAO PH 7,00, FRASCO COM 500 ML"""
682,4,"""HIPOCLORITO DE SODIO."""
683,1,"""CAL HIDRATADA - ESPECIAL PARA TRATAMENTO DE A..."
684,8,"""SULFATO DE ALUMINIO ISENTO DE FERRO LIQUIDO"""
...,...,...
5396043,11,"""BALDE OVAL PRODUZIDO EM POLIPROPILENO, 14 LIT..."
5396044,13,"""BANDEJA EM PAPEL LAMINADA."""
5396045,36,"""ESPONJA PARA USO COZINHA"""
5396046,101,"""VASSOURA DE PÊLO SINTÉTICO"""


In [26]:
df = pd.read_csv("output_dados_abertos/results.csv")

In [28]:
df.loc[(df.cod_item == 200003600936009) & (df.num_ano_referencia == 2020)]

,seq_item_licitacao,seq_licitacao,seq_orgao,num_ano_referencia,num_mes_referencia,num_lote,dsc_lote,cod_item,num_item,dsc_item,dsc_unid_medida,num_versao_arq\n
1192674,22100913,672895,1367,2020,3,-1,"""LOTE NÃO INFORMADO""",200003600936009,1,"""PRESTAÇÃO DE SERVIÇOS - SERVIÇO DE REFORMA D...","""SERV.""","""1""\n"
